In [10]:
# IMPORTO LE LIBRERIE E SCRIVO LE STRINGHE DI AIUTO -------------------
import csv
help_string = "I comandi validi sono i seguenti:\
    \n* aggiungi: aggiungi un prodotto al magazzino\
    \n* elenca: elenca i prodotto in magazzino\
    \n* vendita: registra una vendita effettuata\
    \n* profitti: mostra i profitti totali\
    \n* aiuto: mostra i possibili comandi\
    \n* chiudi: esci dal programma"

error_msg = "I valori validi sono i seguenti:\
    \n* Nome prodotto: stringa\
    \n* Quantità: numero intero\
    \n* P.Acquisto: numero intero o decimale\
    \n* P.Vendita: numero intero o decimale"


### FUNZIONE DI REGISTRAZIONE VENDITA -----------------
def vendita(nome,qt):
    # Ricevo i dati
    l = []
    riga = []
    l2 = []

    with open("database_progettoPAI.csv", "r", encoding="utf-8", newline='') as db:
        reader = csv.reader(db)
        for row in reader:
            l.append(row)
            if nome in row:
                riga = row
                
    with open("Profitti.csv", "r", encoding="utf-8", newline='') as db:
        reader = csv.reader(db)
        for row in reader:
            l2.append(row)
            
    # Modifico i dati
    l2[1][0] = float(l2[1][0]) + qt*float(l[l.index(riga)][3])
    l2[1][1] = float(l2[1][1]) + qt*float(l[l.index(riga)][3]) - qt*float(l[l.index(riga)][2])
    l[l.index(riga)][1] = str(int(float(l[l.index(riga)][1]) - qt))
    
    # Scrivo le modifiche nei rispettivi file
    with open("database_progettoPAI.csv", "w", encoding="utf-8", newline='') as db:
        writer = csv.writer(db)
        for row in l:
            writer.writerow(row)
            
    with open("Profitti.csv", "w", encoding="utf-8", newline='') as db:
        writer = csv.writer(db)
        for row in l2:
            writer.writerow(row)

    subtotal = float(l[l.index(riga)][3])
    vendita.total = qt*subtotal

    print(f"Hai venduto {qt} di {nome} a {subtotal} € cadauno")
    
    
    # IL PROGRAMMA # ----------------------------------------------
cmd = None
l_cmd = ["vendita","aggiungi","elenca","profitti","aiuto","esci","chiudi"]
try:
    while cmd!="esci": 
        # inserisco il metodo .lower per accettare comandi scritti con le maiuscole
        cmd = input("Inserisci un comando: ").lower() 
        
        # Registra una vendita -------------------------------------------
        if cmd=="vendita":
            total_list = []
            continua = None
            
            while continua != "no":
                nome_ven = input("Nome del prodotto: ").lower().capitalize()

                with open("database_progettoPAI.csv", "r", encoding="utf-8", newline='') as db:
                    reader = csv.DictReader(db)
                    lv = []
                    lv_q = []
                    for row in reader:
                        lv.append(row["Nome"])
                        lv_q.append(row["Quantità"])                
                    if nome_ven in lv:
                        print(f"Prodotto presente. Hai {lv_q[lv.index(nome_ven)]} di {nome_ven}")
                        qt_ven = int(input("Quanto ne hai venduto? "))
                        
                        if qt_ven <= int(lv_q[lv.index(nome_ven)]):
                                vendita(nome_ven,qt_ven)
                                total_list.append(vendita.total)
                            
                        else:
                            print("Stai vendendo più di quanto hai in magazzino.\n")
                            ven_esaurimento = input("Vuoi vendere fino ad esaurimento scorte? ").lower()
                            si = ["si","sì","s","yes","y"]
                            if ven_esaurimento in si:
                                qt_ven = int(lv_q[lv.index(nome_ven)])
                                vendita(nome_ven,qt_ven)
                                total_list.append(vendita.total)
                                
                    else:
                        print("Prodotto non presente.\nAggiungilo alla lista, o controlla i prodotti presenti usando il comando: elenca")

                continuare = input("Vuoi vendere un altro prodotto? (si/no)").lower()
                no = ["no","n"]
                si = ["si","sì","s","yes","y"]

                if continuare in no:
                    continua = "no"
                    
            print("Totale vendita: ", sum(total_list), "€")    
        
        ### Aggiungi un nuovo prodotto -------------------------------------------    
        if cmd=="aggiungi":
            nome_agg = input("Nome del prodotto: ").lower().capitalize()         
            with open("database_progettoPAI.csv", "r", encoding="utf-8", newline='') as db:
                reader = csv.reader(db)
                var1 = 0
                l = []
                
                for row in reader:
                    l.append(row)                
                    if nome_agg in row:
                        var1 = 1
                        print("Prodotto presente")
                        qt_agg = int(input("Quantità: "))
                        print(f"Hai aggiunto {nome_agg} x {qt_agg}")
                        l[l.index(row)][1] = int(l[l.index(row)][1]) + qt_agg
                                
                with open("database_progettoPAI.csv", "w", encoding="utf-8", newline='') as db:
                    writer = csv.writer(db)
                    for row in l:
                        writer.writerow(row)
                                        
                db2 = open("database_progettoPAI.csv", "a", encoding="utf-8", newline='')
                writer2 = csv.writer(db2)
                
                if var1 == 0:
                    print("Prodotto non presente")
                    qt_agg2 = int(input("Quantità: "))
                    pa_agg = float(input("Prezzo di acquisto: "))
                    pv_agg = float(input("Prezzo di vendita: "))
                    writer2.writerow([nome_agg,qt_agg2,pa_agg,pv_agg])
                    print(f"Hai aggiunto un nuovo prodotto: {nome_agg}, Quantità: {qt_agg2}, P.Acquisto: {pa_agg} €, P.Vendita: {pv_agg} €")             
                
                db2.close()
        
        
        ### mostra profitti netti e lordi -------------------------------------------
        elif cmd=="profitti":
            with open("Profitti.csv", "r", encoding="utf-8") as db:
                reader = csv.DictReader(db)
                print("%s,%s" % tuple(reader.fieldnames))
                for row in reader:
                    print("%s\t%s\t" % (row["Lordo"],row["Netto"]))
                    
        ### elenca tutti i prodotti nel magazzino -------------------------------------      
        elif cmd=="elenca":
            with open("database_progettoPAI.csv", "r", encoding="utf-8") as db:
                reader = csv.DictReader(db)
                print("%s,%s,%s,%s" % tuple(reader.fieldnames))
                for row in reader:
                    print("%s\t%s\t%s\t%s\t" % (row["Nome"],row["Quantità"],
                                                row["P.Acquisto"],row["P.Vendita"]))

        ### mostra i possibili comandi  -------------------------------------   
        elif cmd=="aiuto":
            print(help_string)
            
        ### saluta e interrompi il programma -------------------------------------
        elif cmd=="chiudi" or cmd =="esci":
            print("Hai chiuso il programma. Bye bye!")
            cmd = "esci"
            
        ### Comando non valido      -------------------------------------
        ### Mostra messaggio di aiuto 
        elif cmd not in l_cmd:
            print("Comando non valido")
            print(help_string)

except:
    print("ValueError\n", error_msg, "\nRiavvia il programma e non fare errori")

Inserisci un comando: vendita
Nome del prodotto: Yogurt
Prodotto presente. Hai 7 di Yogurt
Quanto ne hai venduto? 8
Stai vendendo più di quanto hai in magazzino.

Vuoi vendere fino ad esaurimento scorte? n
Vuoi vendere un altro prodotto? (si/no)n
Totale vendita:  0 €
Inserisci un comando: elenca
Nome,Quantità,P.Acquisto,P.Vendita
Soia	10	5.0	8.0	
Sale	40	5.4	6.0	
Tofu	7	5.2	6.0	
Pasta	10	4.0	9.0	
Salatini	4	5.0	6.0	
Salmone	2	2.0	2.0	
Salame	27	10.0	13.0	
Yogurt	7	20.7	27.8	
Inserisci un comando: esci
Hai chiuso il programma. Bye bye!
